# LDPFL on the CIFAR-10 Dataset

This notebook shows how to run LDPFL on the CIFAR-10 Dataset.

#### Importing packages

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torch.autograd import Variable
from torch.utils.data import Dataset, TensorDataset

import numpy as np
import matplotlib.pyplot as plt
import time
import math
import statistics
from scipy.io import loadmat
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm
from collections import OrderedDict
from collections import defaultdict
from typing import List, Tuple


from CIFAR10_FL import FedMLFunc as fl
from CIFAR10_FL import Net as Net
fl = fl()

from LDP_Functions import SRR
from LDP_Functions import LDP_FL
from LDP_Functions import GRR

print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)

In [ ]:
# Checking the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Getting GPU usage information before computation
if device.type == 'cuda':
    for i in range(0,torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        print('Memory Usage of device :', i)
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#### Main Configuration

In [ ]:
np.random.seed(25)
num_classes = 10 

# Client training settings
epochs = 50 # The number of epochs for the clients during FL  
BATCH_SIZE = 64
weight_decay = 1e-4

# FL settings
num_of_clients = 100
selection_ratio = 0.5
num_rounds = 100 

#### Loading the data and preparing the data

In [ ]:

def load_datasets():
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = CIFAR10("./dataset", train=True, download=True, transform=train_transform)
    test_dataset = CIFAR10("./dataset", train=False, download=True, transform=test_transform)
    
    n_samples = len(train_dataset) // num_of_clients
    class_counts = torch.zeros(10)
    for i in range(len(train_dataset)):
        class_counts[train_dataset[i][1]] += 1

    # Divide the samples for each class into n parts
    class_indices = {}
    for i in range(len(train_dataset)):
        label = train_dataset[i][1]
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(i)

    for label in class_indices:
        np.random.shuffle(class_indices[label])
        class_indices[label] = [class_indices[label][i::num_of_clients] for i in range(num_of_clients)]

    # Create datasets for each client by combining the parts from each class
    datasets_list = []
    for i in range(num_of_clients):
        indices = []
        for label in class_indices:
            indices += class_indices[label][i]
        dataset = torch.utils.data.Subset(train_dataset, indices)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
        datasets_list.append(dataloader)

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

    return datasets_list, test_loader


trainloaders, testloader = load_datasets()

#### Model Summary

In [ ]:
from torchsummary import summary

net = Net()
net.to(device)
summary(net, (3, 32, 32))

#### Declaring the global model and preparing data for federated learning

In [ ]:
# Get rid of optimizers and models, to free up memory
try:
    for o in opt:
        del o
except:
    pass
try:
    for model in client_models:
        del model
except:
    pass
try:
    del global_model
except:
    pass



if device.type == 'cuda':
    for i in range(0,torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        print('Memory Usage of device :', i)
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
torch.cuda.empty_cache()
# Instantiate models and optimizers
global_model = nn.DataParallel(Net()).cuda()
client_models = [nn.DataParallel(Net()) for _ in range(num_of_clients)]
for model in client_models:
    model.load_state_dict(global_model.state_dict())

opt = [optim.Adam(model.parameters(), lr=0.0002) for model in client_models]
# opt = [optim.SGD(model.parameters(), lr=0.004, momentum=0.9, weight_decay=5e-4) for model in client_models]


if device.type == 'cuda':
    for i in range(0,torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        print('Memory Usage of device :', i)
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


In [ ]:

ldp_func = SRR(c = 0.0, r = 0.075, epsilon = 5, delta_d = 30 , precision = 4, m = 10)

# ldp_func = GRR(c = 0.0, r = 0.075, epsilon = 3, precision = 4)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

f1 = plt.figure()
temp = []
for i in range(10):
    for j in range(ldp_func.group_sizes[i]):
        temp.append(ldp_func.probs[i])
dom = ldp_func.data[0.075]
pr = []
for i in range(len(dom)):
    pr.append([dom[i], temp[i]])
pr = sorted(pr, key= lambda x: x[0])
plt.plot([x[0] for x in pr], [x[1] for x in pr], label='Probability Distribution', linestyle = 'solid')
# plt.plot(np.arange(1, num_rounds+1), loss_test_collect, label='Test', linestyle = 'solid')
plt.ylabel('Probability')
plt.xlabel('Domain')
plt.grid(True)
plt.legend()

plt.savefig('probdist3.pdf', format='pdf')

#### Federated Learning 

In [ ]:
LDP_FUNCS = {
    "[0.0, 0.3]" : SRR(c = 0.0, r = 0.3, epsilon = 10, delta_d = 0 , precision = 4, m = 10),
    "[0.0, 0.2]" : SRR(c = 0.0, r = 0.2, epsilon = 10, delta_d = 0 , precision = 4, m = 10),
    "[0.0, 0.075]" : SRR(c = 0.0, r = 0.075, epsilon = 10, delta_d = 0 , precision = 4, m = 10),
    "[0.0, 0.075, 5]" : SRR(c = 0.0, r = 0.075, epsilon = 10, delta_d = 0 , precision = 5, m = 10),
    "[0.0, 0.05]" : SRR(c = 0.0, r = 0.05, epsilon = 10, delta_d = 0 , precision = 4, m = 10),
    "[0.0, 0.05, 5]" : SRR(c = 0.0, r = 0.05, epsilon = 10, delta_d = 0 , precision = 5, m = 10),
    "[0.0, 0.03, 5]" : SRR(c = 0.0, r = 0.03, epsilon = 10, delta_d = 0 , precision = 5, m = 10),
#     "[0.0, 0.04]" : SRR(c = 0.0, r = 0.04, epsilon = 4, delta_d = 0 , precision = 4, m = 10),
#     "[0.0, 0.04, 5]" : SRR(c = 0.0, r = 0.04, epsilon = 4, delta_d = 0 , precision = 5, m = 10),
    "[1.0, 0.1]" : SRR(c = 1.0, r = 0.1, epsilon = 10, delta_d = 0 , precision = 4, m = 10),
    "[1.15, 0.05]" : SRR(c = 1.15, r = 0.05, epsilon = 10, delta_d = 0 , precision = 4, m = 10)
}

In [ ]:
# Adaptive Ranges

ranges = {
    "module.conv_layers.0.weight" : LDP_FUNCS["[0.0, 0.3]"],
    "module.conv_layers.0.bias" : LDP_FUNCS["[0.0, 0.2]"],
    "module.conv_layers.1.weight" : LDP_FUNCS["[1.0, 0.1]"],
    "module.conv_layers.1.bias" : LDP_FUNCS["[0.0, 0.075]"],
    "module.conv_layers.3.weight" : LDP_FUNCS["[0.0, 0.075, 5]"],
    "module.conv_layers.3.bias" : LDP_FUNCS["[0.0, 0.075]"],
    "module.conv_layers.4.weight" : LDP_FUNCS["[1.0, 0.1]"],
    "module.conv_layers.4.bias" : LDP_FUNCS["[0.0, 0.05]"],
    "module.conv_layers.7.weight" : LDP_FUNCS["[0.0, 0.075]"],
    "module.conv_layers.7.bias" : LDP_FUNCS["[0.0, 0.075]"],
    "module.conv_layers.8.weight" : LDP_FUNCS["[1.0, 0.1]"],
    "module.conv_layers.8.bias" : LDP_FUNCS["[0.0, 0.075]"],
    "module.conv_layers.10.weight" : LDP_FUNCS["[0.0, 0.075]"],
    "module.conv_layers.10.bias" : LDP_FUNCS["[0.0, 0.05, 5]"],
    "module.conv_layers.11.weight" : LDP_FUNCS["[1.0, 0.1]"],
    "module.conv_layers.11.bias" : LDP_FUNCS["[0.0, 0.2]"],
    "module.fc_layers.0.weight" : LDP_FUNCS["[0.0, 0.03, 5]"],
    "module.fc_layers.0.bias" : LDP_FUNCS["[0.0, 0.03, 5]"],
    "module.fc_layers.1.weight" : LDP_FUNCS["[1.15, 0.05]"],
    "module.fc_layers.1.bias" : LDP_FUNCS["[0.0, 0.075]"],
    "module.fc_layers.3.weight" : LDP_FUNCS["[0.0, 0.2]"],
    "module.fc_layers.3.bias" : LDP_FUNCS["[0.0, 0.05]"],
#     "module.features5.2.weight" : LDP_FUNCS["[0.0, 0.05, 5]"],
#     "module.features5.2.bias" : LDP_FUNCS["[0.0, 0.05, 5]"],
#     "module.features5.4.weight" : LDP_FUNCS["[0.0, 0.05, 5]"],
#     "module.features5.4.bias" : LDP_FUNCS["[0.0, 0.05, 5]"],
#     "module.classifier.0.weight" : LDP_FUNCS["[0.0, 0.075, 5]"],
#     "module.classifier.0.bias" : LDP_FUNCS["[0.0, 0.075, 5]"],
#     "module.classifier.3.weight" : LDP_FUNCS["[0.0, 0.05]"],
#     "module.classifier.3.bias" : LDP_FUNCS["[0.0, 0.05]"],
#     "module.classifier.6.weight" : LDP_FUNCS["[0.0, 0.075]"],
#     "module.classifier.6.bias" : LDP_FUNCS["[0.0, 0.05]"]
}

In [ ]:
acc_train_collect = []
acc_test_collect = []
loss_train_collect = []
loss_test_collect = []

with open('cifar-10-experiment.txt', 'a+') as f:
    f.write('Experiment with 100 clients, 0.5 selection rate and noise-free\n\n')

index = 0

for r in tqdm(range(num_rounds)):
    
    index += 1
    
    # select clients randomly
    client_idx = np.random.permutation(num_of_clients)[:int(selection_ratio*num_of_clients)]
    
    trainloss = 0
    trainacc = 0
    loss = 0
    count = 0
    
    qwerty = 0
    
    for i in client_idx:  
        count += 1
        client_models[i].cuda()
        opt_state = opt[i].state_dict()
        # new_opt = optim.SGD(client_models[i].parameters(), lr=0.004, momentum=0.9, weight_decay=5e-4)
        new_opt = optim.Adam(client_models[i].parameters(), lr=0.0002)
        new_opt.load_state_dict(opt_state)
        [loss,acc]= fl.client_update(client_models[i], new_opt, trainloaders[i], epochs, device)    
        client_models[i].to('cpu')
        opt[i].load_state_dict(new_opt.state_dict())
        # print(f"Training Loss: {loss} \t Training Accuracy: {acc}")
        
        trainloss += loss
        trainacc += acc
        qwerty += 1
        torch.cuda.empty_cache()
        print(f"{qwerty}. Client {i} Train accuracy : {acc}\n")
        
        
    # server aggregate
    
    start = time.time()
    
    #  fl.server_aggregate(global_model, [client_models[i] for i in client_idx], client_models)
    fl.server_aggregate_srr_adaptive(global_model, [client_models[i] for i in client_idx], client_models, ranges)
    
    end = time.time()
    
    print(f"Time taken for server aggregation : {(end-start)/60} minutes.")
    
    test_loss, test_acc = fl.test(global_model, testloader, device)
    
    print(f"Round {index}")
    print('Avg Train loss %0.3g - Avg Train accuracy: %0.3g  - Test loss %0.3g - Test accuracy: %0.3f' % (trainloss / len(client_idx), trainacc / len(client_idx), test_loss, test_acc))
    
    with open('cifar-10-experiment.txt', 'a+') as f:
        f.write(f"Round{index}: Test Loss : {test_loss} | Test Accuracy: {test_acc}\n")
    
    acc_train_collect.append(trainacc / len(client_idx))
    acc_test_collect.append(test_acc)
    loss_train_collect.append(trainloss / len(client_idx))
    loss_test_collect.append(test_loss)
    
    print("---------------------------------------------------------\n")
    
if device.type == 'cuda':
    for i in range(0,torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        print('Memory Usage of device :', i)
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

print("Training and Evaluation completed!") 

In [ ]:
for name, param in global_model.named_parameters():
    if param.requires_grad:
        print(name, "\n---\n", param.data)
        print('\n\nNEXT LAYER\n\n')

In [ ]:
# TEMPORARY TESTING SCRIPT BELOW

# torch.cuda.empty_cache()
# # Instantiate models and optimizers
# global_model = nn.DataParallel(Net()).cuda()
# client_models = [nn.DataParallel(Net()) for _ in range(50)]

# round_no = 14
# for i, model in enumerate(client_models):
#     client_models[i].load_state_dict(torch.load(f"models/round_{round_no+1}_model_{i}.pt"))

# start = time.time()

# #     cm = np.asarray(client_models)
# fl.server_aggregate(global_model, client_models, client_models)
# #     fl.server_aggregate(global_model, client_models)
# end = time.time()

# print(f"Time taken for server aggregation : {end-start} seconds.")

# # test_loss, test_acc = fl.test(global_model, testloader, device)

# print(f"Round {round_no}")
# # print('Test loss %0.3g - Test accuracy: %0.3f' % (test_loss, test_acc))

#### Plotting

In [ ]:
# %matplotlib inline

# # Plotting loss
# f1 = plt.figure()
# plt.plot(np.arange(1, num_rounds+1), loss_train_collect, label='Train', linestyle = 'dashed')
# plt.plot(np.arange(1, num_rounds+1), loss_test_collect, label='Test', linestyle = 'solid')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.grid(True)
# plt.legend()

# # Plotting accuracy
# f2 = plt.figure()
# plt.plot(np.arange(1, num_rounds+1), acc_train_collect, label='Train', linestyle = 'dashed')
# plt.plot(np.arange(1, num_rounds+1), acc_test_collect, label='Test', linestyle = 'solid')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.grid(True)
# plt.legend()

In [ ]:
# # GPU usage information after computation
# if device.type == 'cuda':
#     for i in range(0,torch.cuda.device_count()):
#         print(torch.cuda.get_device_name(i))
#         print('Memory Usage of device :', i)
#         print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
#         print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB') 

In [ ]:
# TESTING SCRIPT FOR LATENCY OF SRR ON SMALL-VGG


# import random


# ldp_func = SRR(c = 0.0, r = 0.03, epsilon = 5, delta_d = 30 , precision = 5, m = 10)

# start = time.time()
# templist = []
# for i in range(2000000):
#     tempo = ldp_func.perturb(random.uniform(0, 0.075))
# #     tempo = LDP_FL(0.0)
# #     templist.append(tempo)
# #     print(tempo)
# # ldp_func.np_perturb(np.zeros(20490))
# end = time.time()

# # print(f"Mean Value is {statistics.mean(templist)}.")
# del templist

# print(f"Time taken for 2 million perturbs: {end-start} seconds.")

# model = nn.DataParallel(Net()).cuda()
# start = time.time()
# for name, param in model.named_parameters():
# #                     param.data = torch.tensor(list(map(ldp_funcs[name].perturb, param.data.detach().cpu().flatten()))).reshape(param.shape).to(param.device)
#     print(name)
#     print(param.shape)
#     t1 = time.time()
#     param_val = param.data.detach().cpu().flatten().numpy()
# #                     param_val = param.data.detach().flatten()
#     t2 = time.time()
#     print(f"\nTime taken for param detach and flatten : {t2-t1} seconds.")
# #                     perturbed_weight = [ldp_funcs[name].perturb(val) for val in param_val]
#     for idx in range(len(param_val)):
#         param_val[idx] = ldp_func.perturb(param_val[idx])
#     t3 = time.time()
#     print(f"Time taken for perturbing param : {t3-t2} seconds.")
# # #                     perturbed_weight = ldp_func.np_perturb(param.data.detach().cpu().flatten())
# #                     param.data = torch.tensor(perturbed_weight).reshape(param.shape).to(param.device)
#     param.data = torch.tensor(param_val).reshape(param.shape).to(param.device)
# #                     param.data = torch.tensor(perturbed_weight).reshape(param.shape)
#     t4 = time.time()
#     print(f"Time taken for reshape and assign : {t4-t3} seconds.\n")
# #                     print("One param done!")
# #                     print(name, "\n")
# end = time.time()
# print(f"Model {i+1} perturbed!")
# print(f"Time taken for perturbation : {end-start} seconds.\n")